In [1]:
from PIL import Image, ImageDraw, ImageFont
from PIL.ImageOps import invert
import matplotlib.pyplot as plt
import numpy as np
import math
import csv
import os
import codecs
from scipy.spatial import distance

In [2]:
letters = [
'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ё', 'Ж',
'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О',
'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц',
'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю',
'Я']



PATH_font = "Arial Italic.ttf"
size, font_size = 600, 500

In [3]:
def create_folder(dir, folder_name):
    output_dir = os.path.join(dir, folder_name)

    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    return output_dir

In [4]:
def create_image_from_letter(font_path, size_1: int, size_letter: int, letter, dir_normal_letters_imgs, dir_negative_letters_imgs):
  image = Image.new('RGB', (size_1, size_1), color=(255, 255, 255))
  font = ImageFont.truetype(font_path, size=size_letter)

  draw = ImageDraw.Draw(image)

  draw.text((size_1//2, size_1//2), letter, font=font, fill=(0, 0, 0), anchor="mm")
  image.save(os.path.join(dir_normal_letters_imgs, f'{letter}.bmp'))
  img = invert(image)
  img.save(os.path.join(dir_negative_letters_imgs, f'{letter}.bmp'))

In [5]:
def monochrome(input_array):
    height, width = input_array.shape[:2]
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for y in range(height):
        for x in range(width):
            output_array[y][x] = np.mean(input_array[y][x])

    return output_array


In [6]:
def binarization_b1w0(input_array):
    height, width = input_array.shape[:2]
    input_array1 = monochrome(input_array)

    output_array = np.zeros((height, width), int)

    for i in range(height):
        for j in range(width):
            if input_array1[i][j] <= 200:
                output_array[i][j] = 1
            else:
                output_array[i][j] = 0

    return output_array


def binarized(img):
    return np.array(img) > 255//2

In [7]:
def first_nonzero(arr):
    first_non0 = -1
    for i in range (np.shape(arr)[0]):
        if arr[i] > 0:
            first_non0 = i
            break

    return first_non0

def last_nonzero(arr):
    arr_reversed = np.flip(arr)
    first_non0 = -1

    for i in range (np.shape(arr)[0]):
        if arr_reversed[i] > 0:
            first_non0 = np.shape(arr)[0] - i-1
            break
    return first_non0

In [8]:
def profile_y(input_array):
    output_array2 = input_array
    return {'y_axis' : np.arange(0, output_array2.shape[0]).astype(int),
            'values' : np.sum(output_array2, axis=1)
            }


def profile_x(input_array):
    output_array1 = input_array
    return {'values' : np.sum(output_array1, axis=0),
            'x_axis' : np.arange(0, output_array1.shape[1]).astype(int)
            }

In [9]:
def edit_img(img):
    img_array = np.array(img)

    img_array_01 = binarization_b1w0(img_array)
    img_array_cut = img_array_01[first_nonzero( profile_y(img_array_01)['values'] ) : last_nonzero( profile_y(img_array_01)['values'] ),
                                 first_nonzero( profile_x(img_array_01)['values'] ) : last_nonzero( profile_x(img_array_01)['values'] )]

    return img_array_cut

In [10]:
def get_characteristics(img_edited):


    weight = img_edited.sum()

    rel_weight = weight / (img_edited.shape[0] * img_edited.shape[1])

    x_avg = 0
    for x, column in enumerate(img_edited.T):
        x_avg += sum((x + 1) * column)
    x_avg = x_avg / weight

    y_avg = 0
    for y, row in enumerate(img_edited):
        y_avg += sum((y + 1) * row)
    y_avg = y_avg / weight


    rel_x_avg = (x_avg - 1) / (img_edited.shape[1] - 1)
    rel_y_avg = (y_avg - 1) / (img_edited.shape[0] - 1)


    iner_x = 0
    for y, row in enumerate(img_edited):
        iner_x += sum(row*(y + 1 - y_avg)**2)

    iner_y = 0
    for x, column in enumerate(img_edited.T):
        iner_y += sum(column*(x + 1 - x_avg)** 2 )


    rel_iner_x = iner_x / (img_edited.shape[0]** 2 * img_edited.shape[1]** 2)
    rel_iner_y = iner_y / (img_edited.shape[0]** 2 * img_edited.shape[1]** 2)

    return {
        'weight': weight,
        'rel_weight': rel_weight,
        'center': [x_avg, y_avg],
        'rel_center': [rel_x_avg, rel_y_avg],
        'inertia': [iner_x, iner_y],
        'rel_inertia': [rel_iner_x, rel_iner_y]
    }

In [11]:
def save_characteristics_to_csv(letter, characteristics_list, output_file):


    with open(output_file, 'a', newline='') as csvfile:
        fields = [ 'Вес_чёрного', 'Норм_вес_чёрного', 'Центр_массы', 'Нормированный_центр_массы', 'Моменты_инерции', 'Нормированные_моменты_инерции']

        csvwriter = csv.DictWriter(csvfile, fieldnames=fields)
        csvwriter.writeheader()

        center_mass_x, center_mass_y = characteristics_list['center']
        normalized_center_mass_x, normalized_center_mass_y = characteristics_list['rel_center']
        normalized_center_mass_x, normalized_center_mass_y = characteristics_list['rel_center']
        normalized_inertia_x, normalized_inertia_y = characteristics_list['rel_inertia']
        inertia_x, inertia_y = characteristics_list['inertia']

        csvwriter.writerow({

            'Вес_чёрного': f"{characteristics_list['weight']}",
            'Норм_вес_чёрного': f"{characteristics_list['rel_weight']}",
            'Центр_массы': f"({center_mass_x}, {center_mass_y})",
            'Нормированный_центр_массы': f"({normalized_center_mass_x}, {normalized_center_mass_y})",
            'Моменты_инерции': f"({inertia_x}, {inertia_y})",
            'Нормированные_моменты_инерции': f"({normalized_inertia_x}, {normalized_inertia_y})"
            })

In [12]:
def compare(letter, letters, characteristics_list, text_characteristics, comparison_list):
    for letter1 in letters:
        comparison_list[letter][letter1] = {
            'rel_weight': 1 - abs(characteristics_list[letter1]['rel_weight'] - text_characteristics['rel_weight']),
            'rel_center': 1 - distance.euclidean(characteristics_list[letter1]['rel_center'], text_characteristics['rel_center']),
            'rel_inertia': 1 - distance.euclidean(characteristics_list[letter1]['rel_inertia'], text_characteristics['rel_inertia'])
        }



In [13]:
def count_same_letters(result):
    total_letters = len(result)
    same_letters = 0

    for letter, values in result.items():
        if values['weight'] == values['center'] == values['inertia'] == values['mult']:
            same_letters += 1

    percentage = (same_letters / total_letters) * 100
    return percentage


In [14]:
def main():
    DIR_cwd = os.getcwd()
    DIR_output = create_folder(DIR_cwd, "output")
    DIR_normal_letters_imgs = create_folder(DIR_output, "normal")
    DIR_negative_letters_imgs = create_folder(DIR_output, "negative")
    characteristics_list = {}

    for letter4 in letters:
        create_image_from_letter(PATH_font, size, font_size, letter4, DIR_normal_letters_imgs, DIR_negative_letters_imgs)

        PATH_image = os.path.join(DIR_negative_letters_imgs, f'{letter4}.bmp')
        image = Image.open(PATH_image)
        img_array_b0w1 = edit_img(image)

        characteristics_list[f'{letter4}'] = get_characteristics(img_array_b0w1)

    comparison_list = {}
    result_file = open("result.txt", "w")

    for letter in letters:
        create_image_from_letter(PATH_font, 550, 450, letter, DIR_normal_letters_imgs, DIR_negative_letters_imgs)

        input_image = Image.open(os.path.join(DIR_negative_letters_imgs, f'{letter}.bmp'))
        input_array = np.array(input_image)
        input_array1 = edit_img(input_array)
        text_charcteristics = get_characteristics(input_array1)

        comparison_list[f'{letter}'] = {}
        for letter1 in letters:
            comparison_list[f'{letter}'][f'{letter1}'] = {
                'rel_weight': 1 - abs(characteristics_list[f'{letter1}']['rel_weight'] - text_charcteristics['rel_weight']),
                'rel_center': 1 - distance.euclidean(characteristics_list[f'{letter1}']['rel_center'], text_charcteristics['rel_center']),
                'rel_inertia': 1 - distance.euclidean(characteristics_list[f'{letter1}']['rel_inertia'], text_charcteristics['rel_inertia'])
            }

            result_file.write(f"{letter} {letter1} {comparison_list[f'{letter}'][f'{letter1}']}\n")

    result_file.close()

    result = {}
    for letter3 in letters:
        max_similarity = 0
        max_letter = ''

        for letter0 in letters:
            similarity = comparison_list[f'{letter3}'][f'{letter0}']['rel_weight'] * comparison_list[f'{letter3}'][f'{letter0}']['rel_center'] * comparison_list[f'{letter3}'][f'{letter0}']['rel_inertia']
            if similarity > max_similarity:
                max_similarity = similarity
                max_letter = letter0

        result[letter3] = max_letter

    letters_str = ''.join(letters)
    result_str = ''.join(result.values())
    match_count = sum(1 for a, b in zip(letters_str, result_str) if a == b)
    match_percentage = (match_count / len(letters)) * 100


    with open("output.txt", "w") as output_file:
        output_file.write(f"Буквы: {letters_str}\n")
        output_file.write(f"Результат: {result_str}\n")
        output_file.write(f"Процент одинаковых букв: {match_percentage:.2f}%")

    print("Результаты записаны в файл 'output.txt'.")

if __name__ == '__main__':
    main()

Результаты записаны в файл 'output.txt'.
